In [21]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_circles

In [22]:
# 选择图片和标签数据
target = 1
data_name = ['0618', '0854', '1066'][target - 1]
img_path = f'../input_data/{data_name}.png'  # 原图路径
output_features_filename = f'./Features/{data_name}_KPCA_features.npy'  # 数据输出路径

img = cv2.imread(img_path)
image = img.reshape(-1, 3)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_tensor = torch.tensor(image, dtype=torch.float32).to(device)

In [23]:
mean = image_tensor.mean(dim=0)
std = image_tensor.std(dim=0)
image_tensor_scaled = (image_tensor - mean) / std

In [27]:
def rbf_kernel(X, gamma=1.0):
    sq_dists = torch.cdist(X, X, p=2)**2
    return torch.exp(-gamma * sq_dists)

In [28]:
gamma = 10
K = rbf_kernel(image_tensor_scaled, gamma)

N = K.size(0)
one_N = torch.ones(N, N).to(device) / N
K_centered = K - torch.matmul(one_N, K) - torch.matmul(K, one_N) \
            + torch.matmul(one_N, K, one_N)

eigvals, eigvecs = torch.linalg.eig(K_centered)

eigcals_sorted, indices = torch.sort(eigvals, descending=True)
top_n_eigvals = eigvals[:, indices[:,2]].real

X_kpca = torch.matmul(K_centered, top_n_eigvals)

print(X_kpca)

OutOfMemoryError: CUDA out of memory. Tried to allocate 58.21 GiB. GPU 0 has a total capacity of 6.00 GiB of which 4.98 GiB is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 10.49 MiB is allocated by PyTorch, and 11.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)